In [1]:
!apt-get update
!apt-get install -y poppler-utils
!pip install pdf2image python-docx pillow paddlepaddle paddleocr

from paddleocr import PPStructure, PaddleOCR
from docx import Document
from docx.shared import Inches
from pdf2image import convert_from_path
from PIL import Image, ImageDraw, ImageFont, ImageEnhance
import os
import json
import time
from google.colab import drive

def render_table_to_image(table, output_path_base):
    rows = len(table.rows)
    cols = len(table.columns)
    cell_width, cell_height = 600, 120  # Increased cell size for better readability
    max_rows_per_image = 15  # Reduce to ensure better handling of large tables
    image_paths = []

    for start_row in range(0, rows, max_rows_per_image):
        end_row = min(start_row + max_rows_per_image, rows)
        img_height = (end_row - start_row) * cell_height + 150  # More padding
        img_width = cols * cell_width + 150  # More padding

        img = Image.new('RGB', (img_width, img_height), color=(255, 255, 255))
        draw = ImageDraw.Draw(img)

        try:
            font = ImageFont.truetype("arial.ttf", 40)  # Larger font
        except:
            font = ImageFont.load_default(size=40)
            print("Warning: Using default font with size 40.")

        for i, row in enumerate(table.rows[start_row:end_row]):
            for j, cell in enumerate(row.cells):
                x0 = j * cell_width + 25
                y0 = i * cell_height + 25
                x1 = x0 + cell_width
                y1 = y0 + cell_height
                draw.rectangle([x0, y0, x1, y1], outline="black", width=2)  # Thicker outline
                text = cell.text.strip()
                if text:
                    max_width = cell_width - 50  # More margin for text
                    wrapped_text = []
                    words = text.split()
                    current_line = ""
                    for word in words:
                        if draw.textlength(current_line + " " + word, font=font) <= max_width:
                            current_line += " " + word if current_line else word
                        else:
                            wrapped_text.append(current_line)
                            current_line = word
                    if current_line:
                        wrapped_text.append(current_line)
                    for k, line in enumerate(wrapped_text):
                        draw.text((x0 + 10, y0 + 10 + k * 45), line, fill="black", font=font)  # Adjusted line spacing
                    print(f"Drawing text in table cell [{start_row + i},{j}]: {text}")

        img = ImageEnhance.Contrast(img).enhance(5.5)  # Higher contrast
        img = ImageEnhance.Sharpness(img).enhance(5.0)  # Higher sharpness
        img = ImageEnhance.Brightness(img).enhance(1.4)  # More brightness
        output_path = f"{output_path_base}_part{start_row // max_rows_per_image}.png"
        img.save(output_path, dpi=(1500, 1500))  # Higher DPI
        image_paths.append(output_path)
        print(f"Saved rendered table image: {output_path}")

    return image_paths

# Update convert_docx_to_images
def convert_docx_to_images(docx_path, output_folder):
    doc = Document(docx_path)
    images = []
    image_counter = 0

    for para in doc.paragraphs:
        if para.text.strip():
            output_path = f"{output_folder}/para_{image_counter}.png"
            img = Image.new('RGB', (800, 200), color=(255, 255, 255))
            draw = ImageDraw.Draw(img)
            font = ImageFont.load_default()
            draw.text((10, 10), para.text, fill="black", font=font)
            img.save(output_path)
            images.append(output_path)
            print(f"Saved paragraph image: {output_path}")
            image_counter += 1

    for table in doc.tables:
        print(f"Processing table with {len(table.rows)} rows and {len(table.columns)} columns")
        output_path_base = f"{output_folder}/table_{image_counter}"
        table_images = render_table_to_image(table, output_path_base)
        images.extend(table_images)
        image_counter += 1

    embedded_image_paths = extract_embedded_images(docx_path, output_folder)
    images.extend(embedded_image_paths)

    return images

def extract_embedded_images(docx_path, output_folder):
    """Extract embedded images from a DOCX file."""
    doc = Document(docx_path)
    image_paths = []
    image_counter = 0

    for rel in doc.part.rels.values():
        if "image" in rel.reltype:
            img = rel.target_part.blob
            img_path = f"{output_folder}/embedded_image_{image_counter}.png"
            with open(img_path, "wb") as f:
                f.write(img)
            img_pil = Image.open(img_path).convert('RGB')
            img_pil = ImageEnhance.Contrast(img_pil).enhance(4.0)
            img_pil = ImageEnhance.Sharpness(img_pil).enhance(4.0)
            img_pil = ImageEnhance.Brightness(img_pil).enhance(1.2)
            img_pil.save(img_path, dpi=(600, 600))
            image_paths.append(img_path)
            print(f"Extracted and enhanced embedded image: {img_path}")
            image_counter += 1

    return image_paths

def convert_docx_to_images(docx_path, output_folder):
    doc = Document(docx_path)
    images = []
    image_counter = 0

    for para in doc.paragraphs:
        if para.text.strip():
            output_path = f"{output_folder}/para_{image_counter}.png"
            img = Image.new('RGB', (800, 200), color=(255, 255, 255))
            draw = ImageDraw.Draw(img)
            font = ImageFont.load_default()
            draw.text((10, 10), para.text, fill="black", font=font)
            img.save(output_path)
            images.append(output_path)
            print(f"Saved paragraph image: {output_path}")
            image_counter += 1

    for table in doc.tables:
        print(f"Processing table with {len(table.rows)} rows and {len(table.columns)} columns")
        output_path_base = f"{output_folder}/table_{image_counter}"
        table_images = render_table_to_image(table, output_path_base)
        images.extend(table_images)
        image_counter += 1

    embedded_image_paths = extract_embedded_images(docx_path, output_folder)
    images.extend(embedded_image_paths)

    return images

def process_images_with_paddleocr(image_paths, ocr_engine):
    all_content = []

    for image_path in image_paths:
        print(f"Processing image: {image_path}")
        if not os.path.exists(image_path):
            print(f"Error: Image file not found: {image_path}")
            continue

        # Use PPStructure for layout detection
        result = structure_engine(image_path)
        print(f"Structure result for {image_path}: {result}")

        for block in result:
            if not isinstance(block, dict):
                print(f"Skipping non-dict block: {block}")
                continue
            block_type = block.get('type', 'unknown')
            res = block.get('res', {})
            content = ""

            if block_type == 'table':
                print(f"Table detected in {image_path}, res: {res}")
                # Force raw OCR with enhanced settings
                ocr_result = ocr_engine.ocr(image_path, cls=True, det=True, rec=True)  # Removed rec_char_type
                print(f"Raw OCR result for table {image_path}: {ocr_result}")
                if ocr_result and ocr_result[0] is not None and len(ocr_result[0]) > 0:
                    lines = [line[1][0] for line in ocr_result[0] if line[1][0] and line[1][0].strip()]
                    if lines:
                        # Clean and structure table content
                        structured_content = []
                        current_row = []
                        for line in lines:
                            line = line.strip().replace("\n", " ").replace("\r", " ").replace("\t", " ").replace("  ", " ")
                            # Basic cleaning for typos and garbled text
                            line = line.encode('ascii', 'ignore').decode('ascii')  # Remove non-ASCII characters
                            if not line:
                                continue
                            # Try to identify table structure (e.g., numbered lines or "No./Description")
                            if line.startswith("No.") or line.startswith(r"\d+\.\d+") or "Description" in line.upper():
                                if current_row:
                                    structured_content.append("\t".join([r.strip() for r in current_row if r.strip()]))
                                current_row = [line]
                            elif line and current_row:
                                # Split long lines into potential columns if they contain keywords
                                if any(keyword in line.upper() for keyword in ["MUST", "SHOULD", "ENSURE", "SYSTEM"]):
                                    current_row.append(line)
                                else:
                                    current_row[-1] += " " + line
                        if current_row:
                            structured_content.append("\t".join([r.strip() for r in current_row if r.strip()]))
                        content = "\n".join(structured_content) if structured_content else "\n".join(lines)
                        # Apply basic typo correction if needed
                        content = content.replace("are to", "and software to").replace("angucial;", "and increased").replace("Rort high", "support high")
                    else:
                        content = "No text detected"
                else:
                    content = "No text detected"
            else:
                if isinstance(res, list):
                    content = " ".join([item.get('text', '').strip() for item in res if item.get('text')])
                elif isinstance(res, dict):
                    content = res.get('text', '').strip()
                else:
                    content = str(res).strip()

            if content.strip() and content != "No text detected":
                all_content.append({
                    'block_type': block_type,
                    'content': content.strip()
                })

    return all_content

def process_document(file_path, temp_image_folder='/content/temp_images'):
    """Process a document by converting it to images and applying OCR."""
    if not os.path.exists(temp_image_folder):
        os.makedirs(temp_image_folder)

    if file_path.lower().endswith('.pdf'):
        image_paths = convert_pdf_to_images(file_path, temp_image_folder)
    elif file_path.lower().endswith('.docx'):
        image_paths = convert_docx_to_images(file_path, temp_image_folder)
    else:
        print(f"Unsupported file type: {file_path}")
        return []

    content = process_images_with_paddleocr(image_paths, ocr_engine)
    print(f"Generated images: {image_paths}")
    return content

def render_list_to_image(list_items, output_path):
    """Render a list to an image using PIL."""
    img_height = len(list_items) * 30
    img_width = 800
    img = Image.new('RGB', (img_width, img_height), color=(255, 255, 255))
    draw = ImageDraw.Draw(img)
    font = ImageFont.load_default()
    for i, item in enumerate(list_items):
        draw.text((10, i * 30), f"• {item.text}", fill="black", font=font)
    img.save(output_path)

def convert_pdf_to_images(pdf_path, output_folder):
    """Convert a PDF file to PNG images."""
    images = convert_from_path(pdf_path, output_folder=output_folder, fmt='png')
    image_paths = [f"{output_folder}/page_{i}.png" for i in range(len(images))]
    for i, image in enumerate(images):
        image.save(image_paths[i], 'PNG')
    return image_paths

# Mount Google Drive and initialize engines
drive.mount('/content/drive', force_remount=True)
structure_engine = PPStructure(show_log=True, lang='en')  # Ensure English model
ocr_engine = PaddleOCR(use_angle_cls=True, lang='en')  # Ensure English model

# Directory containing documents
documents_dir = '/content/drive/My Drive/lifesciences/training_documents'
output_file_path = '/content/drive/My Drive/lifesciences/organized_content_with_sections.json'

# Process each document
all_documents_content = []
for filename in os.listdir(documents_dir):
    if filename.endswith('.pdf') or filename.endswith('.docx'):
        file_path = os.path.join(documents_dir, filename)
        print(f"Processing file: {filename}")
        document_content = process_document(file_path, '/content/drive/My Drive/lifesciences/images')
        all_documents_content.append({
            'filename': filename,
            'content': document_content
        })

# Save the organized content to a JSON file
with open(output_file_path, 'w') as f:
    json.dump(all_documents_content, f, indent=4)

print(f"Processing complete. Results saved to {output_file_path}.")

Streaming output truncated to the last 5000 lines.
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]], dtype=uint8), 'res': {'cell_bbox': [[17.166868209838867, 22.092119216918945, 191.5923614501953, 57.220069885253906], [926.150146484375, 22.23036003112793, 1831.438720703125, 116.05204010009766], [21.10898780822754, 133.560546875, 161.17745971679688, 189.0656280517578], [931.7052001953125, 122.75448608398438, 1588.16357421875, 229.03363037109375], [22.35275650024414, 249.94676208496094, 166.15548706054688, 310.6617431640625], [904.3572387695312, 250.7394561767578, 1798.965576171875, 356.449462890625], [23.008926391601562, 37

In [ ]:
import gc
import torch  # If you're using PyTorch

# Clear memory
gc.collect()

# If using PyTorch, also clear CUDA memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [1]:
!pip install faiss-cpu transformers torch

  Using cached faiss_cpu-1.10.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (4.4 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached faiss_cpu-1.10.0-cp311-cp311-manylinux_2_28_x86_64.whl (30.7 MB)
Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl (363.4 MB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl (127.9 MB)
Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl (207.5 MB)
  Attempting uninstall: nvidia-cusparse-cu12
    Found existing installation: nvidia-cusparse-cu12 1

In [2]:
import json
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Load the JSON file
json_path = '/content/drive/My Drive/lifesciences/organized_content_with_sections.json'
with open(json_path, 'r') as f:
    data = json.load(f)

# Post-process the JSON data
processed_texts = []
for doc in data:
    filename = doc['filename']
    for block in doc['content']:
        if 'content' in block and block['content'].strip() and block['content'] != "No text detected":
            # Normalize text: lowercase, remove extra whitespace, and clean special characters
            text = block['content'].lower().strip().replace('\n', ' ').replace('\t', ' ').replace('  ', ' ')
            processed_texts.append({
                'filename': filename,
                'block_type': block['block_type'],
                'text': text
            })

# Remove duplicates based on text content
unique_texts = []
seen_texts = set()
for item in processed_texts:
    text = item['text']
    if text not in seen_texts:
        seen_texts.add(text)
        unique_texts.append(item)

print(f"Processed {len(unique_texts)} unique text blocks from {len(data)} documents.")

# Load a pre-trained model for embeddings (e.g., Sentence-BERT)
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to generate embeddings
def get_embeddings(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

# Generate embeddings for all unique texts
texts = [item['text'] for item in unique_texts]
embeddings = get_embeddings(texts)

# Create a FAISS index (using L2 distance for similarity search)
dimension = embeddings.shape[1]  # Embedding dimension
index = faiss.IndexFlatL2(dimension)  # Simple L2 distance index (exact search)
index.add(embeddings)

# Save the FAISS index to disk (optional, for reuse)
faiss.write_index(index, "/content/drive/My Drive/lifesciences/faiss_index.faiss")

print(f"FAISS index created with {len(unique_texts)} vectors.")

# Run a query
query = "What are the key findings in life sciences research?"
query_embedding = get_embeddings([query.lower().strip()])[0]  # Generate embedding for the query
query_embedding = np.array([query_embedding], dtype=np.float32)

# Perform similarity search (k=5 nearest neighbors)
k = 5
distances, indices = index.search(query_embedding, k)

# Print the results
print("\nQuery Results:")
for i, idx in enumerate(indices[0]):
    if idx < len(unique_texts):  # Ensure the index is valid
        result = unique_texts[idx]
        print(f"\nResult {i+1} (Distance: {distances[0][i]:.4f}):")
        print(f"Filename: {result['filename']}")
        print(f"Block Type: {result['block_type']}")
        print(f"Text: {result['text'][:500]}...")  # Show first 500 characters to keep it concise

# Optionally, save the processed data for reference
with open('/content/drive/My Drive/lifesciences/processed_texts.json', 'w') as f:
    json.dump(unique_texts, f, indent=4)

print(f"Processing complete. Results and FAISS index saved.")

Mounted at /content/drive
Processed 134 unique text blocks from 7 documents.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

FAISS index created with 134 vectors.

Query Results:

Result 1 (Distance: 28.2373):
Filename: ProtonGlow_Test_URS (1).docx
Block Type: table
Text: 5.90 lockout/tagout procedures must be established and followed to manage 5.91 aadarircatednelgy dumng hanrenahceancerespe. safety-related functiors and 5.92 thsunaghipe'satafety sarttres ensure thefr effectiveness and 5.93 beanasattauditetyust be ryfansdto evaluate the 5.94 must be implemented to ensure personnel are 5.95 nssaaoasdenthgency 5.96 5.97 sefsonrertugharnndures 5.98 mushhe'natauedgo prevent nnnsua pue anewnp ah!yoewe 5.99 must be included to identify and report system failures that ...

Result 2 (Distance: 28.2412):
Filename: ProtonGlow_Test_URS (1).docx
Block Type: table
Text: 5.45 a comprehensive risk assessment must be conducted to identify and 5.46 maganepterdiagazastge in 5.47 safiyrstandatast dangerous 5.48 medrantagoas reedcures must be established and . followed to manag. 5.49 aadarlcatsd sefgy dumng 5.50 thsunaghipe'sa